In [1]:
# Importing libraries and pysb model
import sys
sys.path.append("..")
import numpy as np
from pydream.core import run_dream
from pysb.simulator import ScipyOdeSimulator
from pydream.parameters import SampledParam
from scipy.stats import norm, uniform
from pydream.convergence import Gelman_Rubin
from jnk3_noask1.model_analysis.jnk3_no_ask1 import model
import pandas as pd
from jnk3_noask1.model_analysis.equilibration_function import pre_equilibration

/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Loading experimental data and initializing PySB solver
exp_data = pd.read_csv('../data/exp_data_arrestin_normalization_1h_138max.csv')

ignore = 1
tspan = np.linspace(0, exp_data['Time (secs)'].values[-(ignore+1)], 121)
t_exp_mask = [idx in exp_data['Time (secs)'].values[:-1] for idx in tspan]
solver = ScipyOdeSimulator(model, tspan=tspan)

In [3]:
# Assigning a normal distribution to each experimental data point. 
# where the distribution mean and std are the mean and std of the experimental data

like_mkk4_arrestin_pjnk3 = norm(loc=exp_data['pTyr_arrestin_avg'].values[:-ignore] + np.finfo(float).eps,
                                scale=exp_data['pTyr_arrestin_std'].values[:-ignore])
like_mkk7_arrestin_pjnk3 = norm(loc=exp_data['pThr_arrestin_avg'].values[:-ignore] + np.finfo(float).eps,
                                scale=exp_data['pThr_arrestin_std'].values[:-ignore])

like_mkk4_noarrestin_pjnk3 = norm(loc=exp_data['pTyr_noarrestin_avg'].values[:-ignore] + np.finfo(float).eps,
                                scale=exp_data['pTyr_noarrestin_std'].values[:-ignore])
like_mkk7_noarrestin_pjnk3 = norm(loc=exp_data['pThr_noarrestin_avg'].values[:-ignore] + np.finfo(float).eps,
                                scale=exp_data['pThr_noarrestin_std'].values[:-ignore])

In [4]:
# Add PySB rate parameters to be sampled as unobserved random variables to DREAM with normal priors

# Indices of the parameters that are going to be calibrated
idx_pars_calibrate = [1, 15, 17, 19, 24, 25, 26, 27]  
rates_of_interest_mask = [i in idx_pars_calibrate for i, par in enumerate(model.parameters)]

arrestin_idx = [32]
jnk3_initial_value = 0.6
jnk3_initial_idxs = [35, 36, 37]

param_values = np.array([p.value for p in model.parameters])

sampled_parameter_names = [SampledParam(norm, loc=np.log10(par), scale=2) for par in param_values[rates_of_interest_mask]]
# We calibrate the pMKK4 - Arrestin-3 reverse reaction rate. We have experimental data
# for this interaction and know that the k_r varies from 160 to 1068 (standard deviation)
sampled_parameter_names[0] = SampledParam(uniform, loc=np.log10(160), scale=np.log10(1068)-np.log10(160))

nchains = 5
niterations = 50000

In [5]:
# likelihood function

def likelihood(position):
    Y = np.copy(position)
    param_values[rates_of_interest_mask] = 10 ** Y

    pars1 = np.copy(param_values)
    pars2 = np.copy(param_values)
    # Pre-equilibration
    time_eq = np.linspace(0, 30, 30)
    pars_eq1 = np.copy(param_values)
    pars_eq2 = np.copy(param_values)

    pars_eq2[arrestin_idx] = 0
    pars_eq2[jnk3_initial_idxs] = [0.492, 0.108, 0]

    all_pars = np.stack((pars_eq1, pars_eq2))
    all_pars[:, [24, 25]] = 0  # Setting catalytic reactions to zero for pre-equilibration
    eq_conc = pre_equilibration(model, time_eq, all_pars)[1]

    # Simulating models with initials from pre-equilibration and parameters for condition with/without arrestin
    pars2[arrestin_idx] = 0
    pars2[jnk3_initial_idxs] = [0.492, 0.108, 0]
    sim = solver.run(param_values=[pars1, pars2], initials=eq_conc).all
    logp_mkk4_arrestin = np.sum(like_mkk4_arrestin_pjnk3.logpdf(sim[0]['pTyr_jnk3'][t_exp_mask] / jnk3_initial_value))
    logp_mkk7_arrestin = np.sum(like_mkk7_arrestin_pjnk3.logpdf(sim[0]['pThr_jnk3'][t_exp_mask] / jnk3_initial_value))

    # No arrestin simulations/experiments

    logp_mkk4_noarrestin = np.sum(like_mkk4_noarrestin_pjnk3.logpdf(sim[1]['pTyr_jnk3'][t_exp_mask] / jnk3_initial_value))
    logp_mkk7_noarrestin = np.sum(like_mkk7_noarrestin_pjnk3.logpdf(sim[1]['pThr_jnk3'][t_exp_mask] / jnk3_initial_value))

    #If model simulation failed due to integrator errors, return a log probability of -inf.
    logp_total = logp_mkk4_arrestin + logp_mkk7_arrestin + logp_mkk4_noarrestin + logp_mkk7_noarrestin
    if np.isnan(logp_total):
        logp_total = -np.inf

    return logp_total


In [ ]:
# Run DREAM sampling.  Documentation of DREAM options is in Dream.py.
converged = False
total_iterations = niterations
sampled_params, log_ps = run_dream(parameters=sampled_parameter_names, likelihood=likelihood,
                                   niterations=niterations, nchains=nchains, multitry=False,
                                   gamma_levels=4, adapt_gamma=True, history_thin=1,
                                   model_name='jnk3_dreamzs_5chain', verbose=True)

# Save sampling output (sampled parameter values and their corresponding logps).
for chain in range(len(sampled_params)):
    np.save('pydream_results/jnk3_dreamzs_5chain_sampled_params_chain_' + str(chain)+'_'+str(total_iterations), sampled_params[chain])
    np.save('pydream_results/jnk3_dreamzs_5chain_logps_chain_' + str(chain)+'_'+str(total_iterations), log_ps[chain])

#Check convergence and continue sampling if not converged

GR = Gelman_Rubin(sampled_params)
print('At iteration: ',total_iterations,' GR = ',GR)
np.savetxt('pydream_results/jnk3_dreamzs_5chain_GelmanRubin_iteration_'+str(total_iterations)+'.txt', GR)

old_samples = sampled_params
if np.any(GR>1.2):
    starts = [sampled_params[chain][-1, :] for chain in range(nchains)]
    while not converged:
        total_iterations += niterations
        sampled_params, log_ps = run_dream(parameters=sampled_parameter_names, likelihood=likelihood,
                                           niterations=niterations, nchains=nchains, start=starts, multitry=False, gamma_levels=4,
                                           adapt_gamma=True, history_thin=1, model_name='jnk3_dreamzs_5chain',
                                           verbose=True, restart=True)


        # Save sampling output (sampled parameter values and their corresponding logps).
        for chain in range(len(sampled_params)):
            np.save('pydream_results/jnk3_dreamzs_5chain_sampled_params_chain_' + str(chain)+'_'+str(total_iterations), sampled_params[chain])
            np.save('pydream_results/jnk3_dreamzs_5chain_logps_chain_' + str(chain)+'_'+str(total_iterations), log_ps[chain])

        old_samples = [np.concatenate((old_samples[chain], sampled_params[chain])) for chain in range(nchains)]
        GR = Gelman_Rubin(old_samples)
        print('At iteration: ',total_iterations,' GR = ',GR)
        np.savetxt('pydream_results/jnk3_dreamzs_5chain_GelmanRubin_iteration_' + str(total_iterations)+'.txt', GR)

        if np.all(GR<1.2):
            converged = True

try:
    #Plot output
    import seaborn as sns
    from matplotlib import pyplot as plt
    total_iterations = len(old_samples[0])
    burnin = total_iterations/2
    samples = np.concatenate((old_samples[0][burnin:, :], old_samples[1][burnin:, :], old_samples[2][burnin:, :],
                              old_samples[3][burnin:, :], old_samples[4][burnin:, :]))

    ndims = len(sampled_parameter_names)
    colors = sns.color_palette(n_colors=ndims)
    for dim in range(ndims):
        fig = plt.figure()
        sns.distplot(samples[:, dim], color=colors[dim], norm_hist=True)
        fig.savefig('pydream_results/PyDREAM_jnk3_dimension_'+str(dim))

except ImportError:
    pass

('Iteration: ', 0, ' acceptance rate: ', 0.0)
('Iteration: ', 0, ' acceptance rate over last 100 iterations: ', 0.0)
History file not loaded.
('Seeding history with ', 80, ' draws from prior.')
('Iteration: ', 0, ' acceptance rate: ', 0.0)
('Iteration: ', 0, ' acceptance rate over last 100 iterations: ', 0.0)
('Iteration: ', 0, ' acceptance rate: ', 0.0)
('Iteration: ', 0, ' acceptance rate over last 100 iterations: ', 0.0)
('Iteration: ', 0, ' acceptance rate: ', 0.0)
('Iteration: ', 0, ' acceptance rate over last 100 iterations: ', 0.0)
('Iteration: ', 0, ' acceptance rate: ', 0.0)
Setting crossover probability starting values.
('Iteration: ', 0, ' acceptance rate over last 100 iterations: ', 0.0)
('Set probability of different crossover values to: ', [0.3333333333333333, 0.3333333333333333, 0.3333333333333333])
Setting start to random draw from prior.
('Start: ', array([ 2.48990669,  3.25764267,  1.2882733 ,  2.85486947, -0.74706347,
        0.607244  ,  1.79619495,  1.67973756]))
H

('Iteration: ', 10, ' acceptance rate: ', 0.6363636363636364)
('Did not accept point.  Kept old logp: ', -119.8760322160388, ' Tested logp: ', -2654.639287691493, ' at temperature: ', 1.0)
('Iteration: ', 10, ' acceptance rate: ', 0.2727272727272727)
('Accepted point.  New logp: ', -13.191314171877409, ' old logp: ', -22.65665977392518, ' at temperature: ', 1.0)
('Iteration: ', 10, ' acceptance rate: ', 0.36363636363636365)
('Did not accept point.  Kept old logp: ', -1505.9316922557864, ' Tested logp: ', -11254.675914799502, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -4020.490990131113, ' Tested logp: ', -4026.799424984744, ' at temperature: ', 1.0)
('Iteration: ', 10, ' acceptance rate: ', 0.18181818181818182)
('Did not accept point.  Kept old logp: ', 10.403538078564559, ' Tested logp: ', -146.01854875426847, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -119.8760322160388, ' Tested logp: ', -276.3594599539081, ' at temperature: ', 1.0

('Did not accept point.  Kept old logp: ', -3927.9152128854134, ' Tested logp: ', -4021.74878039535, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -43.00562807715129, ' Tested logp: ', -45.527677262721895, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', 18.72923497019071, ' Tested logp: ', -135.66383802019914, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -120.04943855880934, ' Tested logp: ', -135.57396483887095, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -13.764413225354831, ' Tested logp: ', -29.2170769254069, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -3927.9152128854134, ' Tested logp: ', -3999.1955248593476, ' at temperature: ', 1.0)
('Did not accept point.  Kept old logp: ', -43.00562807715129, ' Tested logp: ', -49.417090492168796, ' at temperature: ', 1.0)
('Accepted point.  New logp: ', 24.476595993357407, ' old logp: ', 18.72923497019071, ' at temperature: '

Process NoDaemonPoolWorker-3:
Process NoDaemonPoolWorker-5:
Traceback (most recent call last):
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/multiprocess/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
Process NoDaemonPoolWorker-4:
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/multiprocess/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/multiprocess/pool.py", line 113, in worker
    result = (True, func(*args, **kwds))
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/multiprocess/pool.py", line 65, in mapstar
Process NoDaemonPoolWorker-2:
    return map(*args)
Traceback (most recent call last):
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/pydream/core.py", line 102, in _sample_dream
    sampled_params[iteratio

    lines = self._print(expr).splitlines()
    eq_conc = pre_equilibration(model, time_eq, all_pars)[1]
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/printing/printer.py", line 257, in _print
  File "../jnk3_noask1/model_analysis/equilibration_function.py", line 28, in pre_equilibration
    return getattr(self, printmethod)(expr, *args, **kwargs)
    return getattr(self, printmethod)(expr, *args, **kwargs)
    solver = ScipyOdeSimulator(model, tspan=time_search, param_values=parameters).run()
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/printing/str.py", line 56, in _print_Add
    t = self._print(term)
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/printing/str.py", line 51, in _print_Add
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/printing/printer.py", line 257, in _print
    return getattr(self, printmethod)(expr, *args, **kwargs)
  File "/Users/dionis

    coeff *= complex(factor)
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/printing/codeprinter.py", line 388, in _print_Mul
    if c < 0:
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/expr.py", line 243, in __complex__
    re, im = result.as_real_imag()
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/expr.py", line 1720, in as_real_imag
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/numbers.py", line 2128, in __lt__
    return (re(self), im(self))
    other = _sympify(other)
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/cache.py", line 93, in wrapper
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/sympify.py", line 387, in _sympify
    retval = cfunc(*args, **kwargs)
  File "/Users/dionisio/anaconda/envs/nicole/lib/python2.7/site-packages/sympy/core/compatibility.py", li